# 日本語解析の共通処理

## MeCabのインストール

In [1]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

## 文書から単語を抽出

今回は使い勝手のために、文書を直接コードに記載している。  
実際にはファイルから読み込むなどの手法を使用することになる。  

In [0]:
# 学習に使用するデータ
# [(タイトル), (文書)]の構造がリストになっている
docs_train = [
['うどん', 
'うどんは、小麦粉を練って長く切った、ある程度の幅と太さを持つ麺またはその料理であり、\
主に日本で食されているものを指すが、過去の日本の移民政策の影響や食のグローバル化の影響により、\
関係各国にも近似な料理が散見される。\
饂飩とも書く。\
細い物などは「冷麦」「素麺」と分けて称することが一般的ではあるが、\
乾麺に関して太さによる規定がある以外は厳密な規定はなく、細い麺であっても「稲庭うどん」の例も存在し、\
厚みの薄い麺も基準を満たせば、乾麺については「きしめん、ひもかわ」と称してよいと規定があり、\
これらもうどんの一種類に含まれる。'],
['おにぎり', 
'おにぎり（御握り）は、ご飯を三角形・俵形・球状などに加圧成型した食べ物である。\
通常は手のひらに載る程度の大きさに作る。「おむすび」や「握り飯」とも呼ばれる。\
保存性・携行性に優れており、手づかみで食べられることから、日本で古くから今日に至るまで携行食や弁当として重宝されている。\
元々は残り飯の保存や携行食として発達したが、その後は常食としてのおにぎりが主流となり、\
現代ではコンビニエンスストアやスーパーマーケットでも販売されている。\
携行する必要がない居酒屋や定食屋でも提供されるほど、日本の食文化に定着している。\
日本のコンビニエンスストアや外食・中食店の海外進出、日本滞在経験を持つ外国人の増加に伴い、\
世界各国でおにぎりが販売されるようになっている。']
]

# テストに使用するデータ
# 上記の学習に使用するデータの中から、このデータに類似したものを探す
doc_test = [
['そうめん', 
'素麺（索麺、そうめん）は、小麦粉を原料とした日本および東アジアの麺のひとつ。\
主に乾麺として流通するため、市場で通年入手できるが、冷やして食することが多く、\
清涼感を求めて夏の麺料理として食するのが一般的である。']
]

In [3]:
import MeCab
from gensim.models.doc2vec import TaggedDocument

from google.colab import files

# MeCabインスタンスを生成
# 単語分解した際の出力フォーマットを"chasen"形式にする
mecab = MeCab.Tagger("-Ochasen")

# docs_trainのタイトルと、文書から抽出した単語一覧のTaggedDocument型リスト
title_words_lists = []

# doc_testのタイトルと、文書から抽出した単語一覧のTaggedDocument型リスト
title_words_lists_test = []

# 文書に含まれる単語を抽出する関数
# 引数には文字列型を指定する
def extractWords(content):
  # 文章から抽出した単語の一覧
  words = []

  # 文章を単語に分解(単語ごとに1行の分解結果が出力される)
  lines = mecab.parse(content).splitlines()
  for line in lines:
    chunks = line.split('\t') # 分解結果の項目はタブで区切られている
    # TODO: 抽出する品詞は要調整
    # 分解結果を確認し、動詞・形容詞・名詞(数を除く)のみ抽出
    if len(chunks) > 3 \
        and (chunks[3].startswith('動詞') or chunks[3].startswith('形容詞') \
             or (chunks[3].startswith('名詞') and not chunks[3].startswith('名詞-数'))):
      words.append(chunks[0])
  return words


print('===単語抽出実施(学習データ)===')
for doc in docs_train:
  title = doc[0]
  content = doc[1]

  # 文章から単語を抽出
  words = extractWords(content)
  # タイトルと、そこから抽出した単語一覧をセットにして登録
  title_words_lists.append(TaggedDocument(words=words, tags=[title]))
  
  # 単語一覧を出力
  print(title, words)

print('===単語抽出実施(テストデータ)===')
for doc in doc_test:
  title = doc[0]
  content = doc[1]

  # 文章から単語を抽出
  words = extractWords(content)
  # タイトルと、そこから抽出した単語一覧をセットにして登録
  title_words_lists_test.append(TaggedDocument(words=words, tags=[title]))
  
  # 単語一覧を出力
  print(title, words)

===単語抽出実施(学習データ)===
うどん ['うどん', '小麦粉', '練っ', '長く', '切っ', '幅', '太', 'さ', '持つ', '麺', '料理', '主', '日本', '食', 'さ', 'れ', 'いる', 'もの', '指す', '過去', '日本', '移民', '政策', '影響', '食', 'グローバル', '化', '影響', '関係', '各国', '近似', '料理', '散見', 'さ', 'れる', '饂飩', '書く', '細い', '物', '冷麦', '素麺', '分け', '称する', 'こと', '一般', '的', 'ある', '乾麺', '太', 'さ', '規定', 'ある', '以外', '厳密', '規定', '細い', '麺', '稲庭', 'うどん', '例', '存在', 'し', '厚み', '薄い', '麺', '基準', '満たせ', '乾麺', 'きし', 'めん', 'ひも', '称し', 'よい', '規定', 'あり', 'これら', 'うどん', '種類', '含ま', 'れる']
おにぎり ['おにぎり', '握り', 'ご飯', '三角形', '俵', '形', '球状', '加', '圧', '成型', 'し', '食べ物', '通常', '手のひら', '載る', '程度', '大き', 'さ', '作る', 'おむすび', '握り飯', '呼ば', 'れる', '保存', '性', '携行', '性', '優れ', 'おり', '手づかみ', '食べ', 'られる', 'こと', '日本', '今日', '至る', '携行', '食', '弁当', '重宝', 'さ', 'れ', 'いる', '残り', '飯', '保存', '携行', '食', '発達', 'し', 'その後', '常食', 'おにぎり', '主流', 'なり', '現代', 'コンビニエンスストア', 'スーパーマーケット', '販売', 'さ', 'れ', 'いる', '携行', 'する', '必要', 'ない', '居酒屋', '定食', '屋', '提供', 'さ', 'れる', 'ほど', '日本', '食', '文化', '定着', 'し', 'いる', '日本', 'コンビニエン

## 単語抽出結果の確認と精度改善

単語抽出結果から、単語が期待するように分割されているかを確認する。  
Mecab辞書をカスタマイズすることで、単語分割の精度できる可能性がある。  

MeCab: 単語の追加方法  
https://taku910.github.io/mecab/dic.html

また、使用している辞書についても、上記の例ではmecab-ipadicを使用しているが、  
mecab-ipadic-neologdという辞書もあり、こちらの方が新語に強いと言われている。

https://github.com/neologd/mecab-ipadic-neologd


# 分析方法

## TF-IDF
単語の出現回数に着目し、その文章の「特徴」となる単語をスコア付けする方法。  
「特徴」となる単語が共通している文書は似ていると推測される。

In [4]:
# 学習用データ、テスト用データの両方について、各単語のTF-IDF値を算出

from gensim import corpora
from gensim import models
from operator import itemgetter

# 学習データから抽出したすべての単語を集約したリストを作成
all_words = []
for title_words in title_words_lists:
  all_words.append(title_words.words)

# 単語をID化し重複を除外した辞書を作成
dictionary = corpora.Dictionary(all_words)

# 辞書にテストデータの単語を追加
for title_words in title_words_lists_test:
  all_words.append(title_words.words)
dictionary.add_documents(all_words)

# 文章ごとに含まれる単語IDの個数を算出
corpus = list(map(dictionary.doc2bow,all_words))

# TF-IDFモデルの生成
test_model = models.TfidfModel(corpus)

# 文章に含まれる各単語についてTF-IDF値を算出
corpus_tfidf = test_model[corpus]

print('===TF-IDF値表示===')
all_lists = title_words_lists + title_words_lists_test
for (title_words, vector) in zip(all_lists, corpus_tfidf):
  print(title_words.tags) # タイトルを出力
  
  # TF-IDF値の大きい順に単語を並べ替え
  sorted_vector = sorted(vector, key=itemgetter(1), reverse=True)

  # sorted_vectorは[(単語ID), (TF-IDF値)]のリストとなっているため、単語IDを単語に戻す
  for word_tfidf in sorted_vector:
    print('  {0}: {1}'.format(dictionary[word_tfidf[0]], word_tfidf[1]))

===TF-IDF値表示===
['うどん']
  うどん: 0.3397988898947734
  規定: 0.3397988898947734
  ある: 0.22653259326318226
  太: 0.22653259326318226
  影響: 0.22653259326318226
  細い: 0.22653259326318226
  さ: 0.16721288003947896
  麺: 0.1254096600296092
  あり: 0.11326629663159113
  きし: 0.11326629663159113
  これら: 0.11326629663159113
  ひも: 0.11326629663159113
  もの: 0.11326629663159113
  よい: 0.11326629663159113
  グローバル: 0.11326629663159113
  以外: 0.11326629663159113
  例: 0.11326629663159113
  冷麦: 0.11326629663159113
  分け: 0.11326629663159113
  切っ: 0.11326629663159113
  化: 0.11326629663159113
  厚み: 0.11326629663159113
  厳密: 0.11326629663159113
  含ま: 0.11326629663159113
  基準: 0.11326629663159113
  存在: 0.11326629663159113
  幅: 0.11326629663159113
  指す: 0.11326629663159113
  政策: 0.11326629663159113
  散見: 0.11326629663159113
  書く: 0.11326629663159113
  満たせ: 0.11326629663159113
  物: 0.11326629663159113
  称し: 0.11326629663159113
  称する: 0.11326629663159113
  移民: 0.11326629663159113
  種類: 0.11326629663159113
  稲庭: 0.113266296

In [5]:
# 学習用データの中から、テスト用データに類似するものを検索

# テストデータのTF-IDF値を取得(テストデータはリストの末尾)
test_vector = corpus_tfidf[len(corpus_tfidf)-1]

# TF-IDF値の大きい順に単語を並び変え
test_vector_sorted = sorted(vector, key=itemgetter(1), reverse=True)

# テスト用データに含まれる単語が他の文書に存在するか確認
print('===テストデータと同一の単語を含む文書===')
print('単語(テストデータ文書でのTF-IDF値): 単語を含む文書タイトル(その文書でのTF-IDF値) ...')
print('------')
for test in test_vector_sorted:
  print_str = '' # 結果表示用
  for i, target in enumerate(corpus_tfidf):
    # corpus_tfidfの末尾はテストデータ自身なので、確認はその前まで
    if i == len(corpus_tfidf)-1:
      break;
    for target_word in target:
      if test[0] == target_word[0]:
        print_str += ' {0}({1})'.format(title_words_lists[i].tags, target_word[1])
  if print_str != '':
    print('{0}({1}):{2}'.format(dictionary[test[0]], test[1], print_str)) # test[0]は単語IDのため、単語に変換

===テストデータと同一の単語を含む文書===
単語(テストデータ文書でのTF-IDF値): 単語を含む文書タイトル(その文書でのTF-IDF値) ...
------
麺(0.15468571200378836): ['うどん'](0.1254096600296092)
めん(0.07734285600189418): ['うどん'](0.04180322000986974)
一般(0.07734285600189418): ['うどん'](0.04180322000986974)
主(0.07734285600189418): ['うどん'](0.04180322000986974)
乾麺(0.07734285600189418): ['うどん'](0.08360644001973948)
小麦粉(0.07734285600189418): ['うどん'](0.04180322000986974)
料理(0.07734285600189418): ['うどん'](0.08360644001973948)
的(0.07734285600189418): ['うどん'](0.04180322000986974)
素麺(0.07734285600189418): ['うどん'](0.04180322000986974)
する(0.07734285600189418): ['おにぎり'](0.03519105924668807)


TF-IDFは単語の出現回数から統計的に算出されるものであるため、調整する仕組みはない。  
もし調整をするのであれば、特定の単語に対してTF-IDF値を増減するテーブルを作成する程度。  

また、類似度の表現方法について、上記では単語が一致するかどうかで見ているが、  
さらに一致した単語のTF-IDF値などを使ってスコア化すると、どの文書が似ているかがわかりやすくなる。  
※ただし、どの観点で似ているかが見えにくくなることには注意すべき。

## Doc2Vec 
文書を多次元のベクトル値に変換する方法。  
ベクトルの距離で類似度を算出できる他にも、足し算や引き算を行うことができる。  
※A文書とB文書の、両方の要素を持っている文書は？ といった具合。

In [6]:
from gensim.models.doc2vec import Doc2Vec

# 学習実行。引数の意味は以下の通り。(適宜調整する必要がある)
#  documents: 文書のタグと単語リストがセットになったもの(TaggedDocument)のリスト 
#  vector_size:  文書ベクトルの次元数
#  window:  指定した個数分の、隣接する単語を使って学習を行う
#  min_count: 単語の出現回数が、ここで指定した回数に満たない単語を除外する
#  epochs: 学習回数
#  workers: 学習実行に使用するスレッド数
model = Doc2Vec(documents=title_words_lists, vector_size=10, window=3, min_count=1, epochs=5, workers=6)

# 学習結果を保存(学習には時間がかかることがあるため)
model.save('test.model')

# 学習結果を使用して、テスト用データをベクトル化
vector = model.infer_vector(title_words_lists_test[0].words)

# ベクトル化したテスト用データに対して、学習用データから類似するものを表示
similar_texts = model.docvecs.most_similar([vector])
print('===テスト用データに類似する文書===')
print('(タイトル, 近似スコア)')
print('------')
for similar_text in similar_texts:
  print(similar_text)


===テスト用データに類似する文書===
(タイトル, 近似スコア)
------
('おにぎり', 0.10985822230577469)
('うどん', -0.08030092716217041)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


学習はランダム要素があるため、同じコードを使っても近似スコアが異なる。    
類似度のチューニングについては、Doc2Vec関数の引数を調整することで実施できる可能性がある。  
ただし、どの引数を調整してどのように結果が変化するかは試してみないとわからない。  
インプットするデータをきれいにする方がうまくいくこともある。